## Setup

In [ ]:
# Install microdf
!pip install git+https://github.com/PSLmodels/microdf.git
!pip install git+https://github.com/MaxGhenis/scf.git

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import microdf as mdf
import scf

In [ ]:
scf = mdf.read_stata_zip('https://www.federalreserve.gov/econres/files/scfp2019s.zip')
detail = mdf.read_stata_zip('https://www.federalreserve.gov/econres/files/scf2019s.zip')

In [ ]:
scf['numextra'] = (detail.x6533 == 1) + (detail.x6534 == 1)

/usr/local/lib/python3.6/dist-packages/pandas/core/computation/expressions.py:204: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  f"evaluating in Python space because the {repr(op_str)} "


In [ ]:
scf['numper'] = 1 + scf.famstruct.isin([4, 5]) + scf.kids + scf.numextra

In [ ]:
scf['income_pp'] = scf.income / scf.numper
scf['networth_pp'] = scf.networth / scf.numper
# Weight households by amount of people
scf['wgt_numper'] = scf.wgt * scf.numper

In [ ]:
# Income quintiles
mdf.add_weighted_quantiles(scf, 'income_pp', 'wgt_numper')

scf.drop(['income_pp_percentile', 'income_pp_2percentile',
           'income_pp_ventile', 'income_pp_decile',
           'income_pp_quartile'],
          axis=1, inplace=True)

scf.groupby('income_pp_quintile')[['wgt_numper']].sum()

,wgt_numper
income_pp_quintile,
1,6.098401e+07
2,6.100444e+07
3,6.099144e+07
4,6.099438e+07
5,6.101400e+07


In [ ]:
# Networth quintiles
mdf.add_weighted_quantiles(scf, 'networth_pp', 'wgt_numper')

scf.drop(['networth_pp_percentile', 'networth_pp_2percentile',
           'networth_pp_ventile', 'networth_pp_decile',
           'networth_pp_quartile'],
          axis=1, inplace=True)

# combine 0 and 1
scf['networth_pp_quintile2'] = np.where(scf.networth_pp_quintile < 2, 0, scf.networth_pp_quintile)
scf.groupby('networth_pp_quintile2')[['wgt_numper']].sum()

,wgt_numper
networth_pp_quintile2,
0,6.099631e+07
2,6.099471e+07
3,6.098782e+07
4,6.099198e+07
5,6.101745e+07


In [ ]:
# Maybe rename?  '100th doesn't seem right way to name that
scf['networth_pp_quintile2'] = scf['networth_pp_quintile2'].astype(str)
scf['networth_pp_quintile2'].replace({'0': '0-20', '2': '20-40', '3': '40-60',
                                   '4': '60-80', '5': '80-100'},inplace=True)

scf['income_pp_quintile'] = scf['income_pp_quintile'].astype(str)
scf['income_pp_quintile'].replace({'1': '0-20', '2': '20-40', '3': '40-60',
                                   '4': '60-80', '5': '80-100'},inplace=True)

In [ ]:
# Identify race codes
scf['race'] = scf['race'].astype(str)
scf['race'].replace({'1': 'White', '2':'Black',
                     '3': 'Hispanic', '5': 'Other'},inplace=True)

# Identify age codes
scf['agecl'] = scf['agecl'].astype(str)
scf['agecl'].replace({'1':'<35', '2': '35-44', '3': '45-54', '4':'55-64',
                      '5':'65-74', '6':'>=75'},inplace=True)

# Identify education level
scf['edcl'] = scf['edcl'].astype(str)
scf['edcl'].replace({'1': 'Less than high school diploma',
                     '2':'High school diploma', '3': 'Some college',
                     '4': 'College graduate'},inplace=True)

In [ ]:
# Calculate estimated population
population = (scf.numper * scf.wgt).sum()
population / 1e6

304.9882663276286

In [ ]:
scf['has_ed_debt'] = scf.edn_inst > 0
scf['ed_debt_pp'] = scf.edn_inst / scf.numper

In [ ]:
scf['opm_pov_thresh'] = (np.where(scf.numper == 1, 12_490, 
                        np.where(scf.numper == 2, 16_910,
                        np.where(scf.numper == 3, 21_330,
                        np.where(scf.numper == 4, 25_750,
                        np.where(scf.numper == 5, 30_170,
                        np.where(scf.numper == 6, 34_590,
                        np.where(scf.numper == 7, 39_010, 43_430))))))))

In [ ]:
scf['original_poor'] = scf.income < scf.opm_pov_thresh
total_poor = (scf.wgt_numper * scf.original_poor).sum()
percent_poor = (total_poor / population * 100).round(1)
percent_poor

10.0

## Who holds student debt?

In [ ]:
# Calculate total education debt 
total_student_debt = (scf.wgt * scf.edn_inst).sum()
total_student_debt / 1e12

1.1146904950337784

In [ ]:
all = pd.DataFrame([mdf.weighted_mean(scf, 'ed_debt_pp', 'wgt_numper'),
                     mdf.weighted_sum(scf, 'has_ed_debt', 'wgt_numper'),
                     mdf.weighted_sum(scf, 'wgt_numper'),
                     mdf.weighted_sum(scf, 'ed_debt_pp', 'wgt_numper')],
                     index=['mean_debt', 'has_debt', 'population', 'total_debt_held'])

all = all.transpose()
all['percent_has_debt'] = (all.has_debt / all.population * 100).round(1)
all['percent_of_total_debt'] = all.total_debt_held / total_student_debt
all['percent_pop'] = all.population / population
all['ratio'] = (all.percent_of_total_debt / all.percent_pop).round(2)
all['mean_debt'] = all['mean_debt'].astype(int)
all = all.reset_index()

In [ ]:
# By race
race = scf.groupby('race').apply(lambda x: pd.Series(
    [mdf.weighted_mean(x, 'ed_debt_pp', 'wgt_numper'),
     mdf.weighted_sum(x, 'has_ed_debt', 'wgt_numper'),
     mdf.weighted_sum(x, 'wgt_numper'),
     mdf.weighted_sum(x, 'ed_debt_pp', 'wgt_numper')],
    index=['mean_debt', 'has_debt', 'population', 'total_debt_held']))

race['percent_has_debt'] = (race.has_debt / race.population * 100).round(1)
race['percent_of_total_debt'] = race.total_debt_held / total_student_debt
race['percent_pop'] = race.population / population
race['ratio'] = (race.percent_of_total_debt / race.percent_pop).round(2)
race['mean_debt'] = race['mean_debt'].astype(int)

race = race.reset_index()

In [ ]:
# By Education Level
education = scf.groupby('edcl').apply(lambda x: pd.Series(
    [mdf.weighted_mean(x, 'ed_debt_pp', 'wgt_numper'),
     mdf.weighted_sum(x, 'has_ed_debt', 'wgt_numper'),
     mdf.weighted_sum(x, 'wgt_numper'),
     mdf.weighted_sum(x, 'ed_debt_pp', 'wgt_numper')], 
    index=['mean_debt', 'has_debt', 'population', 'total_debt_held']))

education['percent_has_debt'] = (education.has_debt / education.population * 100).round(1)
education['percent_of_total_debt'] = education.total_debt_held / total_student_debt
education['percent_pop'] = education.population / population
education['ratio'] = (education.percent_of_total_debt / education.percent_pop).round(2)
education['mean_debt'] = education['mean_debt'].astype(int)
education = education.reset_index()

In [ ]:
# By age Level
age = scf.groupby('agecl').apply(lambda x: pd.Series(
    [mdf.weighted_mean(x, 'ed_debt_pp', 'wgt_numper'),
     mdf.weighted_sum(x, 'has_ed_debt', 'wgt_numper'),
     mdf.weighted_sum(x, 'wgt_numper'),
     mdf.weighted_sum(x, 'ed_debt_pp', 'wgt_numper')], 
    index=['mean_debt', 'has_debt', 'population', 'total_debt_held']))

age['percent_has_debt'] = (age.has_debt / age.population * 100).round(1)
age['percent_of_total_debt'] = age.total_debt_held / total_student_debt
age['percent_pop'] = age.population / population
age['ratio'] = (age.percent_of_total_debt / age.percent_pop).round(2)
age['mean_debt'] = age['mean_debt'].astype(int)
age = age.reset_index()

In [ ]:
# By income Level
income = scf.groupby('income_pp_quintile').apply(lambda x: pd.Series(
    [mdf.weighted_mean(x, 'ed_debt_pp', 'wgt_numper'),
     mdf.weighted_sum(x, 'has_ed_debt', 'wgt_numper'),
     mdf.weighted_sum(x, 'wgt_numper'),
     mdf.weighted_sum(x, 'ed_debt_pp', 'wgt_numper')], 
    index=['mean_debt', 'has_debt', 'population', 'total_debt_held']))

income['percent_has_debt'] = (income.has_debt / income.population * 100).round(1)
income['percent_of_total_debt'] = income.total_debt_held / total_student_debt
income['percent_pop'] = income.population / population
income['ratio'] = (income.percent_of_total_debt / income.percent_pop).round(2)
income['mean_debt'] = income['mean_debt'].astype(int)
income = income.reset_index()

In [ ]:
# By networth Level
networth = scf.groupby('networth_pp_quintile2').apply(lambda x: pd.Series(
    [mdf.weighted_mean(x, 'ed_debt_pp', 'wgt_numper'),
     mdf.weighted_sum(x, 'has_ed_debt', 'wgt_numper'),
     mdf.weighted_sum(x, 'wgt_numper'),
     mdf.weighted_sum(x, 'ed_debt_pp', 'wgt_numper')], 
    index=['mean_debt', 'has_debt', 'population', 'total_debt_held']))

networth['percent_has_debt'] = (networth.has_debt / networth.population * 100).round(1)
networth['percent_of_total_debt'] = networth.total_debt_held / total_student_debt
networth['percent_pop'] = networth.population / population
networth['ratio'] = (networth.percent_of_total_debt / networth.percent_pop).round(2)
networth['mean_debt'] = networth['mean_debt'].astype(int)
networth = networth.reset_index()

In [ ]:
# By poor Level
poor = scf.groupby('original_poor').apply(lambda x: pd.Series(
    [mdf.weighted_mean(x, 'ed_debt_pp', 'wgt_numper'),
     mdf.weighted_sum(x, 'has_ed_debt', 'wgt_numper'),
     mdf.weighted_sum(x, 'wgt_numper'),
     mdf.weighted_sum(x, 'ed_debt_pp', 'wgt_numper')], 
    index=['mean_debt', 'has_debt', 'population', 'total_debt_held']))

poor['percent_has_debt'] = (poor.has_debt / poor.population * 100).round(1)
poor['percent_of_total_debt'] = poor.total_debt_held / total_student_debt
poor['percent_pop'] = poor.population / population
poor['ratio'] = (poor.percent_of_total_debt / poor.percent_pop).round(2)
poor['mean_debt'] = poor['mean_debt'].astype(int)
poor = poor.reset_index()

## Calculate

In [ ]:
# Calculate new networth with eduction debt cancellation
scf['new_networth_no_ed_debt'] = scf.networth + scf.edn_inst

In [ ]:
# Calculate the ubi per person
ubi = total_student_debt / population
ubi

3654.863541000428

In [ ]:
# Give each household their total UBI
scf['total_ubi'] = ubi * scf.numper

In [ ]:
# Calculate new networth with UBI
scf['new_networth_ubi'] = scf.total_ubi + scf.networth

In [ ]:
# Calculate per person
scf['income_pp'] = scf.income / scf.numper
scf['networth_pp'] = scf.networth / scf.numper
scf['ubi_networth_pp'] = scf.networth_pp + ubi
scf['no_debt_networth_pp'] = scf.networth_pp + scf.ed_debt_pp
scf['ubi_income_pp'] = scf.income_pp + ubi
scf['no_debt_income_pp'] = scf.income_pp + scf.ed_debt_pp

In [ ]:
# Check to make sure reforms cost the same
((scf.new_networth_no_ed_debt * scf.wgt).sum()) - ((scf.new_networth_ubi * scf.wgt).sum()) 

0.0

## Groupings

In [ ]:
def all_fun(index_num):
  target_persons = scf.copy(deep=True)

  # Calculate average benefit by reform
  start = mdf.weighted_mean(target_persons, 'networth_pp', 'wgt_numper')
  ed_debt = mdf.weighted_mean(target_persons, 'no_debt_networth_pp', 'wgt_numper')
  ubi = mdf.weighted_mean(target_persons, 'ubi_networth_pp', 'wgt_numper')

  ed_debt_change = ed_debt - start
  ubi_change = ubi - start

  # Calculate percent better off with ed debt cancellation
  target_persons['better_with_ed'] = target_persons.ubi_networth_pp < target_persons.no_debt_networth_pp
  total_better_off_with_ed = (target_persons.better_with_ed * target_persons.wgt_numper).sum()
  target_pop = target_persons.wgt_numper.sum()
  percent_better_off_with_ed = (total_better_off_with_ed / target_pop * 100).round(1)

  # Calculate percent out of debt
  target_persons['initial_debt'] = target_persons.networth_pp < 0
  target_persons['ubi_debt'] = target_persons.ubi_networth_pp < 0
  target_persons['no_ed_debt_debt'] = target_persons.no_debt_networth_pp < 0

  percent_initial_debt = (((target_persons.initial_debt 
                           * target_persons.wgt_numper).sum()) / target_pop 
                          * 100).round(1)
  percent_ubi_debt = (((target_persons.ubi_debt 
                           * target_persons.wgt_numper).sum()) / target_pop
                      * 100).round(1)
  percent_no_ed_debt_debt = (((target_persons.no_ed_debt_debt 
                           * target_persons.wgt_numper).sum()) / target_pop
                           * 100).round(1)

  # Calculate percent poor
  total_original_poor = (target_persons.original_poor * target_persons.wgt_numper).sum()
  percent_original_poor = (total_original_poor / target_pop * 100).round(1)

  target_persons['ubi_income'] = target_persons.total_ubi + target_persons.income
  target_persons['ubi_poor'] = target_persons.ubi_income < target_persons.opm_pov_thresh
  ubi_total_poor = (target_persons.wgt_numper * target_persons.ubi_poor).sum()
  ubi_percent_poor = ubi_total_poor / target_pop
  percent_ubi_poor = (ubi_percent_poor * 100).round(1)

  target_persons['ed_debt_income'] = target_persons.edn_inst + target_persons.income
  target_persons['ed_debt_poor'] = target_persons.ed_debt_income < target_persons.opm_pov_thresh
  ed_debt_total_poor = (target_persons.wgt_numper * target_persons.ed_debt_poor).sum()
  ed_debt_percent_poor = ed_debt_total_poor / target_pop
  percent_ed_debt_poor = (ed_debt_percent_poor * 100).round(1)

  # Calculate poverty gap
  target_persons['original_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.income
  original_gap_billions = (((target_persons.original_poor * target_persons.original_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ubi_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.ubi_income
  ubi_gap_billions = (((target_persons.ubi_poor * target_persons.ubi_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ed_debt_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.ed_debt_income
  ed_debt_gap_billions = (((target_persons.ed_debt_poor * target_persons.ed_debt_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  # Convert and round
  ed_debt_change = ed_debt_change.astype(int)
  ubi_change = ubi_change.astype(int)

  return pd.Series([ed_debt_change, ubi_change, percent_better_off_with_ed, 
                    percent_initial_debt, percent_ubi_debt, 
                    percent_no_ed_debt_debt, percent_original_poor, 
                    percent_ubi_poor, percent_ed_debt_poor,
                    original_gap_billions, ubi_gap_billions, ed_debt_gap_billions])

In [ ]:
def all_row(row):
  return all_fun(row.index)

In [ ]:
all[['ed_debt_change', 'ubi_change', 'percent_better_off_with_ed',
        'in_debt', 'in_debt_ubi', 'in_debt_no_ed_debt',
        'percent_original_poor', 'percent_ubi_poor', 'percent_ed_debt_poor', 
     'original_gap_billions', 'ubi_gap_billions', 'ed_debt_gap_billions']] = all.apply(all_row, axis=1)

In [ ]:
def race_fun(race):
  target_persons = scf[scf.race==race].copy(deep=True)

  # Calculate average benefit by reform
  start = mdf.weighted_mean(target_persons, 'networth_pp', 'wgt_numper')
  ed_debt = mdf.weighted_mean(target_persons, 'no_debt_networth_pp', 'wgt_numper')
  ubi = mdf.weighted_mean(target_persons, 'ubi_networth_pp', 'wgt_numper')

  ed_debt_change = ed_debt - start
  ubi_change = ubi - start

  # Calculate percent better off with ed debt cancellation
  target_persons['better_with_ed'] = target_persons.ubi_networth_pp < target_persons.no_debt_networth_pp
  total_better_off_with_ed = (target_persons.better_with_ed * target_persons.wgt_numper).sum()
  target_pop = target_persons.wgt_numper.sum()
  percent_better_off_with_ed = (total_better_off_with_ed / target_pop * 100).round(1)

  # Calculate percent out of debt
  target_persons['initial_debt'] = target_persons.networth_pp < 0
  target_persons['ubi_debt'] = target_persons.ubi_networth_pp < 0
  target_persons['no_ed_debt_debt'] = target_persons.no_debt_networth_pp < 0

  percent_initial_debt = (((target_persons.initial_debt 
                           * target_persons.wgt_numper).sum()) / target_pop 
                          * 100).round(1)
  percent_ubi_debt = (((target_persons.ubi_debt 
                           * target_persons.wgt_numper).sum()) / target_pop
                      * 100).round(1)
  percent_no_ed_debt_debt = (((target_persons.no_ed_debt_debt 
                           * target_persons.wgt_numper).sum()) / target_pop
                           * 100).round(1)

  # Calculate percent poor
  total_original_poor = (target_persons.original_poor * target_persons.wgt_numper).sum()
  percent_original_poor = (total_original_poor / target_pop * 100).round(1)

  target_persons['ubi_income'] = target_persons.total_ubi + target_persons.income
  target_persons['ubi_poor'] = target_persons.ubi_income < target_persons.opm_pov_thresh
  ubi_total_poor = (target_persons.wgt_numper * target_persons.ubi_poor).sum()
  ubi_percent_poor = ubi_total_poor / target_pop
  percent_ubi_poor = (ubi_percent_poor * 100).round(1)

  target_persons['ed_debt_income'] = target_persons.edn_inst + target_persons.income
  target_persons['ed_debt_poor'] = target_persons.ed_debt_income < target_persons.opm_pov_thresh
  ed_debt_total_poor = (target_persons.wgt_numper * target_persons.ed_debt_poor).sum()
  ed_debt_percent_poor = ed_debt_total_poor / target_pop
  percent_ed_debt_poor = (ed_debt_percent_poor * 100).round(1)

  # Calculate poverty gap
  target_persons['original_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.income
  original_gap_billions = (((target_persons.original_poor * target_persons.original_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ubi_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.ubi_income
  ubi_gap_billions = (((target_persons.ubi_poor * target_persons.ubi_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ed_debt_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.ed_debt_income
  ed_debt_gap_billions = (((target_persons.ed_debt_poor * target_persons.ed_debt_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  # Convert and round
  ed_debt_change = ed_debt_change.astype(int)
  ubi_change = ubi_change.astype(int)

  return pd.Series([ed_debt_change, ubi_change, percent_better_off_with_ed, 
                    percent_initial_debt, percent_ubi_debt, 
                    percent_no_ed_debt_debt, percent_original_poor, 
                    percent_ubi_poor, percent_ed_debt_poor,
                    original_gap_billions, ubi_gap_billions, ed_debt_gap_billions])

In [ ]:
def race_row(row):
  return race_fun(row.race)

In [ ]:
race[['ed_debt_change', 'ubi_change', 'percent_better_off_with_ed',
        'in_debt', 'in_debt_ubi', 'in_debt_no_ed_debt',
        'percent_original_poor', 'percent_ubi_poor', 'percent_ed_debt_poor', 
     'original_gap_billions', 'ubi_gap_billions', 'ed_debt_gap_billions']] = race.apply(race_row, axis=1)

In [ ]:
def education_fun(edcl):
  target_persons = scf[scf.edcl==edcl].copy(deep=True)

  # Calculate average benefit by reform
  start = mdf.weighted_mean(target_persons, 'networth_pp', 'wgt_numper')
  ed_debt = mdf.weighted_mean(target_persons, 'no_debt_networth_pp', 'wgt_numper')
  ubi = mdf.weighted_mean(target_persons, 'ubi_networth_pp', 'wgt_numper')

  ed_debt_change = ed_debt - start
  ubi_change = ubi - start

  # Calculate percent better off with ed debt cancellation
  target_persons['better_with_ed'] = target_persons.ubi_networth_pp < target_persons.no_debt_networth_pp
  total_better_off_with_ed = (target_persons.better_with_ed * target_persons.wgt_numper).sum()
  target_pop = target_persons.wgt_numper.sum()
  percent_better_off_with_ed = (total_better_off_with_ed / target_pop * 100).round(1)

  # Calculate percent out of debt
  target_persons['initial_debt'] = target_persons.networth_pp < 0
  target_persons['ubi_debt'] = target_persons.ubi_networth_pp < 0
  target_persons['no_ed_debt_debt'] = target_persons.no_debt_networth_pp < 0

  percent_initial_debt = (((target_persons.initial_debt 
                           * target_persons.wgt_numper).sum()) / target_pop 
                          * 100).round(1)
  percent_ubi_debt = (((target_persons.ubi_debt 
                           * target_persons.wgt_numper).sum()) / target_pop
                      * 100).round(1)
  percent_no_ed_debt_debt = (((target_persons.no_ed_debt_debt 
                           * target_persons.wgt_numper).sum()) / target_pop
                           * 100).round(1)

  # Calculate percent poor
  total_original_poor = (target_persons.original_poor * target_persons.wgt_numper).sum()
  percent_original_poor = (total_original_poor / target_pop * 100).round(1)

  target_persons['ubi_income'] = target_persons.total_ubi + target_persons.income
  target_persons['ubi_poor'] = target_persons.ubi_income < target_persons.opm_pov_thresh
  ubi_total_poor = (target_persons.wgt_numper * target_persons.ubi_poor).sum()
  ubi_percent_poor = ubi_total_poor / target_pop
  percent_ubi_poor = (ubi_percent_poor * 100).round(1)

  target_persons['ed_debt_income'] = target_persons.edn_inst + target_persons.income
  target_persons['ed_debt_poor'] = target_persons.ed_debt_income < target_persons.opm_pov_thresh
  ed_debt_total_poor = (target_persons.wgt_numper * target_persons.ed_debt_poor).sum()
  ed_debt_percent_poor = ed_debt_total_poor / target_pop
  percent_ed_debt_poor = (ed_debt_percent_poor * 100).round(1)

  # Calculate poverty gap
  target_persons['original_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.income
  original_gap_billions = (((target_persons.original_poor * target_persons.original_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ubi_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.ubi_income
  ubi_gap_billions = (((target_persons.ubi_poor * target_persons.ubi_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ed_debt_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.ed_debt_income
  ed_debt_gap_billions = (((target_persons.ed_debt_poor * target_persons.ed_debt_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  # Convert and round
  ed_debt_change = ed_debt_change.astype(int)
  ubi_change = ubi_change.astype(int)

  return pd.Series([ed_debt_change, ubi_change, percent_better_off_with_ed, 
                    percent_initial_debt, percent_ubi_debt, 
                    percent_no_ed_debt_debt, percent_original_poor, 
                    percent_ubi_poor, percent_ed_debt_poor,
                    original_gap_billions, ubi_gap_billions, ed_debt_gap_billions])

In [ ]:
def education_row(row):
  return education_fun(row.edcl)

In [ ]:
education[['ed_debt_change', 'ubi_change', 'percent_better_off_with_ed',
        'in_debt', 'in_debt_ubi', 'in_debt_no_ed_debt',
        'percent_original_poor', 'percent_ubi_poor', 'percent_ed_debt_poor', 
     'original_gap_billions', 'ubi_gap_billions', 'ed_debt_gap_billions']] = education.apply(education_row, axis=1)

In [ ]:
def income_fun(income_pp_quintile):
  target_persons = scf[scf.income_pp_quintile==income_pp_quintile].copy(deep=True)

  # Calculate average benefit by reform
  start = mdf.weighted_mean(target_persons, 'networth_pp', 'wgt_numper')
  ed_debt = mdf.weighted_mean(target_persons, 'no_debt_networth_pp', 'wgt_numper')
  ubi = mdf.weighted_mean(target_persons, 'ubi_networth_pp', 'wgt_numper')

  ed_debt_change = ed_debt - start
  ubi_change = ubi - start

  # Calculate percent better off with ed debt cancellation
  target_persons['better_with_ed'] = target_persons.ubi_networth_pp < target_persons.no_debt_networth_pp
  total_better_off_with_ed = (target_persons.better_with_ed * target_persons.wgt_numper).sum()
  target_pop = target_persons.wgt_numper.sum()
  percent_better_off_with_ed = (total_better_off_with_ed / target_pop * 100).round(1)

  # Calculate percent out of debt
  target_persons['initial_debt'] = target_persons.networth_pp < 0
  target_persons['ubi_debt'] = target_persons.ubi_networth_pp < 0
  target_persons['no_ed_debt_debt'] = target_persons.no_debt_networth_pp < 0

  percent_initial_debt = (((target_persons.initial_debt 
                           * target_persons.wgt_numper).sum()) / target_pop 
                          * 100).round(1)
  percent_ubi_debt = (((target_persons.ubi_debt 
                           * target_persons.wgt_numper).sum()) / target_pop
                      * 100).round(1)
  percent_no_ed_debt_debt = (((target_persons.no_ed_debt_debt 
                           * target_persons.wgt_numper).sum()) / target_pop
                           * 100).round(1)

  # Calculate percent poor
  total_original_poor = (target_persons.original_poor * target_persons.wgt_numper).sum()
  percent_original_poor = (total_original_poor / target_pop * 100).round(1)

  target_persons['ubi_income'] = target_persons.total_ubi + target_persons.income
  target_persons['ubi_poor'] = target_persons.ubi_income < target_persons.opm_pov_thresh
  ubi_total_poor = (target_persons.wgt_numper * target_persons.ubi_poor).sum()
  ubi_percent_poor = ubi_total_poor / target_pop
  percent_ubi_poor = (ubi_percent_poor * 100).round(1)

  target_persons['ed_debt_income'] = target_persons.edn_inst + target_persons.income
  target_persons['ed_debt_poor'] = target_persons.ed_debt_income < target_persons.opm_pov_thresh
  ed_debt_total_poor = (target_persons.wgt_numper * target_persons.ed_debt_poor).sum()
  ed_debt_percent_poor = ed_debt_total_poor / target_pop
  percent_ed_debt_poor = (ed_debt_percent_poor * 100).round(1)

  # Calculate poverty gap
  target_persons['original_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.income
  original_gap_billions = (((target_persons.original_poor * target_persons.original_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ubi_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.ubi_income
  ubi_gap_billions = (((target_persons.ubi_poor * target_persons.ubi_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ed_debt_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.ed_debt_income
  ed_debt_gap_billions = (((target_persons.ed_debt_poor * target_persons.ed_debt_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  # Convert and round
  ed_debt_change = ed_debt_change.astype(int)
  ubi_change = ubi_change.astype(int)

  return pd.Series([ed_debt_change, ubi_change, percent_better_off_with_ed, 
                    percent_initial_debt, percent_ubi_debt, 
                    percent_no_ed_debt_debt, percent_original_poor, 
                    percent_ubi_poor, percent_ed_debt_poor,
                    original_gap_billions, ubi_gap_billions, ed_debt_gap_billions])

In [ ]:
def income_row(row):
  return income_fun(row.income_pp_quintile)

In [ ]:
income[['ed_debt_change', 'ubi_change', 'percent_better_off_with_ed',
        'in_debt', 'in_debt_ubi', 'in_debt_no_ed_debt',
        'percent_original_poor', 'percent_ubi_poor', 'percent_ed_debt_poor', 
     'original_gap_billions', 'ubi_gap_billions', 'ed_debt_gap_billions']] = income.apply(income_row, axis=1)

In [ ]:
def age_fun(agecl):
  target_persons = scf[scf.agecl==agecl].copy(deep=True)

  # Calculate average benefit by reform
  start = mdf.weighted_mean(target_persons, 'networth_pp', 'wgt_numper')
  ed_debt = mdf.weighted_mean(target_persons, 'no_debt_networth_pp', 'wgt_numper')
  ubi = mdf.weighted_mean(target_persons, 'ubi_networth_pp', 'wgt_numper')

  ed_debt_change = ed_debt - start
  ubi_change = ubi - start

  # Calculate percent better off with ed debt cancellation
  target_persons['better_with_ed'] = target_persons.ubi_networth_pp < target_persons.no_debt_networth_pp
  total_better_off_with_ed = (target_persons.better_with_ed * target_persons.wgt_numper).sum()
  target_pop = target_persons.wgt_numper.sum()
  percent_better_off_with_ed = (total_better_off_with_ed / target_pop * 100).round(1)

  # Calculate percent out of debt
  target_persons['initial_debt'] = target_persons.networth_pp < 0
  target_persons['ubi_debt'] = target_persons.ubi_networth_pp < 0
  target_persons['no_ed_debt_debt'] = target_persons.no_debt_networth_pp < 0

  percent_initial_debt = (((target_persons.initial_debt 
                           * target_persons.wgt_numper).sum()) / target_pop 
                          * 100).round(1)
  percent_ubi_debt = (((target_persons.ubi_debt 
                           * target_persons.wgt_numper).sum()) / target_pop
                      * 100).round(1)
  percent_no_ed_debt_debt = (((target_persons.no_ed_debt_debt 
                           * target_persons.wgt_numper).sum()) / target_pop
                           * 100).round(1)

  # Calculate percent poor
  total_original_poor = (target_persons.original_poor * target_persons.wgt_numper).sum()
  percent_original_poor = (total_original_poor / target_pop * 100).round(1)

  target_persons['ubi_income'] = target_persons.total_ubi + target_persons.income
  target_persons['ubi_poor'] = target_persons.ubi_income < target_persons.opm_pov_thresh
  ubi_total_poor = (target_persons.wgt_numper * target_persons.ubi_poor).sum()
  ubi_percent_poor = ubi_total_poor / target_pop
  percent_ubi_poor = (ubi_percent_poor * 100).round(1)

  target_persons['ed_debt_income'] = target_persons.edn_inst + target_persons.income
  target_persons['ed_debt_poor'] = target_persons.ed_debt_income < target_persons.opm_pov_thresh
  ed_debt_total_poor = (target_persons.wgt_numper * target_persons.ed_debt_poor).sum()
  ed_debt_percent_poor = ed_debt_total_poor / target_pop
  percent_ed_debt_poor = (ed_debt_percent_poor * 100).round(1)

  # Calculate poverty gap
  target_persons['original_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.income
  original_gap_billions = (((target_persons.original_poor * target_persons.original_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ubi_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.ubi_income
  ubi_gap_billions = (((target_persons.ubi_poor * target_persons.ubi_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ed_debt_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.ed_debt_income
  ed_debt_gap_billions = (((target_persons.ed_debt_poor * target_persons.ed_debt_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  # Convert and round
  ed_debt_change = ed_debt_change.astype(int)
  ubi_change = ubi_change.astype(int)

  return pd.Series([ed_debt_change, ubi_change, percent_better_off_with_ed, 
                    percent_initial_debt, percent_ubi_debt, 
                    percent_no_ed_debt_debt, percent_original_poor, 
                    percent_ubi_poor, percent_ed_debt_poor,
                    original_gap_billions, ubi_gap_billions, ed_debt_gap_billions])

In [ ]:
def age_row(row):
  return age_fun(row.agecl)

In [ ]:
age[['ed_debt_change', 'ubi_change', 'percent_better_off_with_ed',
        'in_debt', 'in_debt_ubi', 'in_debt_no_ed_debt',
        'percent_original_poor', 'percent_ubi_poor', 'percent_ed_debt_poor', 
     'original_gap_billions', 'ubi_gap_billions', 'ed_debt_gap_billions']] = age.apply(age_row, axis=1)

In [ ]:
def networth_fun(networth_pp_quintile2):
  target_persons = scf[scf.networth_pp_quintile2==networth_pp_quintile2].copy(deep=True)

  # Calculate average benefit by reform
  start = mdf.weighted_mean(target_persons, 'networth_pp', 'wgt_numper')
  ed_debt = mdf.weighted_mean(target_persons, 'no_debt_networth_pp', 'wgt_numper')
  ubi = mdf.weighted_mean(target_persons, 'ubi_networth_pp', 'wgt_numper')

  ed_debt_change = ed_debt - start
  ubi_change = ubi - start

  # Calculate percent better off with ed debt cancellation
  target_persons['better_with_ed'] = target_persons.ubi_networth_pp < target_persons.no_debt_networth_pp
  total_better_off_with_ed = (target_persons.better_with_ed * target_persons.wgt_numper).sum()
  target_pop = target_persons.wgt_numper.sum()
  percent_better_off_with_ed = (total_better_off_with_ed / target_pop * 100).round(1)

  # Calculate percent out of debt
  target_persons['initial_debt'] = target_persons.networth_pp < 0
  target_persons['ubi_debt'] = target_persons.ubi_networth_pp < 0
  target_persons['no_ed_debt_debt'] = target_persons.no_debt_networth_pp < 0

  percent_initial_debt = (((target_persons.initial_debt 
                           * target_persons.wgt_numper).sum()) / target_pop 
                          * 100).round(1)
  percent_ubi_debt = (((target_persons.ubi_debt 
                           * target_persons.wgt_numper).sum()) / target_pop
                      * 100).round(1)
  percent_no_ed_debt_debt = (((target_persons.no_ed_debt_debt 
                           * target_persons.wgt_numper).sum()) / target_pop
                           * 100).round(1)

  # Calculate percent poor
  total_original_poor = (target_persons.original_poor * target_persons.wgt_numper).sum()
  percent_original_poor = (total_original_poor / target_pop * 100).round(1)

  target_persons['ubi_income'] = target_persons.total_ubi + target_persons.income
  target_persons['ubi_poor'] = target_persons.ubi_income < target_persons.opm_pov_thresh
  ubi_total_poor = (target_persons.wgt_numper * target_persons.ubi_poor).sum()
  ubi_percent_poor = ubi_total_poor / target_pop
  percent_ubi_poor = (ubi_percent_poor * 100).round(1)

  target_persons['ed_debt_income'] = target_persons.edn_inst + target_persons.income
  target_persons['ed_debt_poor'] = target_persons.ed_debt_income < target_persons.opm_pov_thresh
  ed_debt_total_poor = (target_persons.wgt_numper * target_persons.ed_debt_poor).sum()
  ed_debt_percent_poor = ed_debt_total_poor / target_pop
  percent_ed_debt_poor = (ed_debt_percent_poor * 100).round(1)

  # Calculate poverty gap
  target_persons['original_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.income
  original_gap_billions = (((target_persons.original_poor * target_persons.original_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ubi_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.ubi_income
  ubi_gap_billions = (((target_persons.ubi_poor * target_persons.ubi_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ed_debt_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.ed_debt_income
  ed_debt_gap_billions = (((target_persons.ed_debt_poor * target_persons.ed_debt_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  # Convert and round
  ed_debt_change = ed_debt_change.astype(int)
  ubi_change = ubi_change.astype(int)

  return pd.Series([ed_debt_change, ubi_change, percent_better_off_with_ed, 
                    percent_initial_debt, percent_ubi_debt, 
                    percent_no_ed_debt_debt, percent_original_poor, 
                    percent_ubi_poor, percent_ed_debt_poor,
                    original_gap_billions, ubi_gap_billions, ed_debt_gap_billions])

In [ ]:
def networth_row(row):
  return networth_fun(row.networth_pp_quintile2)

In [ ]:
networth[['ed_debt_change', 'ubi_change', 'percent_better_off_with_ed',
        'in_debt', 'in_debt_ubi', 'in_debt_no_ed_debt',
        'percent_original_poor', 'percent_ubi_poor', 'percent_ed_debt_poor', 
     'original_gap_billions', 'ubi_gap_billions', 'ed_debt_gap_billions']] = networth.apply(networth_row, axis=1)

In [ ]:
def poor_fun(original_poor):
  target_persons = scf[scf.original_poor==original_poor].copy(deep=True)

  # Calculate average benefit by reform
  start = mdf.weighted_mean(target_persons, 'networth_pp', 'wgt_numper')
  ed_debt = mdf.weighted_mean(target_persons, 'no_debt_networth_pp', 'wgt_numper')
  ubi = mdf.weighted_mean(target_persons, 'ubi_networth_pp', 'wgt_numper')

  ed_debt_change = ed_debt - start
  ubi_change = ubi - start

  # Calculate percent better off with ed debt cancellation
  target_persons['better_with_ed'] = target_persons.ubi_networth_pp < target_persons.no_debt_networth_pp
  total_better_off_with_ed = (target_persons.better_with_ed * target_persons.wgt_numper).sum()
  target_pop = target_persons.wgt_numper.sum()
  percent_better_off_with_ed = (total_better_off_with_ed / target_pop * 100).round(1)

  # Calculate percent out of debt
  target_persons['initial_debt'] = target_persons.networth_pp < 0
  target_persons['ubi_debt'] = target_persons.ubi_networth_pp < 0
  target_persons['no_ed_debt_debt'] = target_persons.no_debt_networth_pp < 0

  percent_initial_debt = (((target_persons.initial_debt 
                           * target_persons.wgt_numper).sum()) / target_pop 
                          * 100).round(1)
  percent_ubi_debt = (((target_persons.ubi_debt 
                           * target_persons.wgt_numper).sum()) / target_pop
                      * 100).round(1)
  percent_no_ed_debt_debt = (((target_persons.no_ed_debt_debt 
                           * target_persons.wgt_numper).sum()) / target_pop
                           * 100).round(1)

  # Calculate percent poor
  total_original_poor = (target_persons.original_poor * target_persons.wgt_numper).sum()
  percent_original_poor = (total_original_poor / target_pop * 100).round(1)

  target_persons['ubi_income'] = target_persons.total_ubi + target_persons.income
  target_persons['ubi_poor'] = target_persons.ubi_income < target_persons.opm_pov_thresh
  ubi_total_poor = (target_persons.wgt_numper * target_persons.ubi_poor).sum()
  ubi_percent_poor = ubi_total_poor / target_pop
  percent_ubi_poor = (ubi_percent_poor * 100).round(1)

  target_persons['ed_debt_income'] = target_persons.edn_inst + target_persons.income
  target_persons['ed_debt_poor'] = target_persons.ed_debt_income < target_persons.opm_pov_thresh
  ed_debt_total_poor = (target_persons.wgt_numper * target_persons.ed_debt_poor).sum()
  ed_debt_percent_poor = ed_debt_total_poor / target_pop
  percent_ed_debt_poor = (ed_debt_percent_poor * 100).round(1)

  # Calculate poverty gap
  target_persons['original_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.income
  original_gap_billions = (((target_persons.original_poor * target_persons.original_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ubi_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.ubi_income
  ubi_gap_billions = (((target_persons.ubi_poor * target_persons.ubi_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  target_persons['ed_debt_poverty_gap'] = target_persons.opm_pov_thresh - target_persons.ed_debt_income
  ed_debt_gap_billions = (((target_persons.ed_debt_poor * target_persons.ed_debt_poverty_gap * target_persons.wgt_numper).sum()) / 1e9).round(1)

  # Convert and round
  ed_debt_change = ed_debt_change.astype(int)
  ubi_change = ubi_change.astype(int)

  return pd.Series([ed_debt_change, ubi_change, percent_better_off_with_ed, 
                    percent_initial_debt, percent_ubi_debt, 
                    percent_no_ed_debt_debt, percent_original_poor, 
                    percent_ubi_poor, percent_ed_debt_poor,
                    original_gap_billions, ubi_gap_billions, ed_debt_gap_billions])

In [ ]:
def poor_row(row):
  return poor_fun(row.original_poor)

In [ ]:
poor[['ed_debt_change', 'ubi_change', 'percent_better_off_with_ed',
        'in_debt', 'in_debt_ubi', 'in_debt_no_ed_debt',
        'percent_original_poor', 'percent_ubi_poor', 'percent_ed_debt_poor', 
     'original_gap_billions', 'ubi_gap_billions', 'ed_debt_gap_billions']] = poor.apply(poor_row, axis=1)

## Edit Tables

In [ ]:
education.sort_values(by=['percent_original_poor'], inplace=True,ascending=False)

In [ ]:
age.sort_values(by=['percent_of_total_debt'], inplace=True, ascending=False)

In [ ]:
poor.sort_values(by=['percent_of_total_debt'], inplace=True, ascending=True)

In [ ]:
poor['original_poor'] = poor['original_poor'].astype(str)
poor['original_poor'].replace({'True': 'In poverty', 'False':'Not in poverty'},inplace=True)

In [ ]:
education["edcl"].replace({"Less than high school diploma": ["Less than high<br>school diploma"],
                           "High school diploma": "High school<br>diploma",
                          "Some college": "Some<br>college",
                          "College graduate": "College<br>graduate"}, inplace=True)

In [ ]:
all['index'] = all['index'].replace([0],'All')

In [ ]:
all = all.rename(columns={"index": "race"})
race = pd.concat([race, all])

In [ ]:
all = all.rename(columns={"race": "edcl"})
education = pd.concat([education, all])

In [ ]:
all = all.rename(columns={"edcl": "income_pp_quintile"})
income = pd.concat([income, all])

In [ ]:
all = all.rename(columns={"income_pp_quintile": "agecl"})
age = pd.concat([age, all])

In [ ]:
all = all.rename(columns={"agecl": "networth_pp_quintile2"})
networth = pd.concat([networth, all])

In [ ]:
all = all.rename(columns={"networth_pp_quintile2": "original_poor"})
poor = pd.concat([poor, all])

In [ ]:
race = race.astype({"ubi_change":'int', "ed_debt_change":'int'})
education = education.astype({"ubi_change":'int', "ed_debt_change":'int'})
age = age.astype({"ubi_change":'int', "ed_debt_change":'int'})
income = income.astype({"ubi_change":'int', "ed_debt_change":'int'})
networth = networth.astype({"ubi_change":'int', "ed_debt_change":'int'})
poor = poor.astype({"ubi_change":'int', "ed_debt_change":'int'})

In [ ]:
race['mean_debt'] = race.apply(lambda x: "{:,}".format(x['mean_debt']), axis=1)
race['ed_debt_change'] = race.apply(lambda x: "{:,}".format(x['ed_debt_change']), axis=1)
race['ubi_change'] = race.apply(lambda x: "{:,}".format(x['ubi_change']), axis=1)

In [ ]:
education['mean_debt'] = education.apply(lambda x: "{:,}".format(x['mean_debt']), axis=1)
education['ed_debt_change'] = education.apply(lambda x: "{:,}".format(x['ed_debt_change']), axis=1)
education['ubi_change'] = education.apply(lambda x: "{:,}".format(x['ubi_change']), axis=1)

In [ ]:
age['mean_debt'] = age.apply(lambda x: "{:,}".format(x['mean_debt']), axis=1)
age['ed_debt_change'] = age.apply(lambda x: "{:,}".format(x['ed_debt_change']), axis=1)
age['ubi_change'] = age.apply(lambda x: "{:,}".format(x['ubi_change']), axis=1)

In [ ]:
income['mean_debt'] = income.apply(lambda x: "{:,}".format(x['mean_debt']), axis=1)
income['ed_debt_change'] = income.apply(lambda x: "{:,}".format(x['ed_debt_change']), axis=1)
income['ubi_change'] = income.apply(lambda x: "{:,}".format(x['ubi_change']), axis=1)

In [ ]:
networth['mean_debt'] = networth.apply(lambda x: "{:,}".format(x['mean_debt']), axis=1)
networth['ed_debt_change'] = networth.apply(lambda x: "{:,}".format(x['ed_debt_change']), axis=1)
networth['ubi_change'] = networth.apply(lambda x: "{:,}".format(x['ubi_change']), axis=1)

In [ ]:
poor['mean_debt'] = poor.apply(lambda x: "{:,}".format(x['mean_debt']), axis=1)
poor['ed_debt_change'] = poor.apply(lambda x: "{:,}".format(x['ed_debt_change']), axis=1)
poor['ubi_change'] = poor.apply(lambda x: "{:,}".format(x['ubi_change']), axis=1)

In [ ]:
race

,race,mean_debt,has_debt,population,total_debt_held,percent_has_debt,percent_of_total_debt,percent_pop,ratio,ed_debt_change,ubi_change,percent_better_off_with_ed,in_debt,in_debt_ubi,in_debt_no_ed_debt,percent_original_poor,percent_ubi_poor,percent_ed_debt_poor,original_gap_billions,ubi_gap_billions,ed_debt_gap_billions
0,Black,"5,882",1.466812e+07,4.412936e+07,2.595805e+11,33.2,0.232872,0.144692,1.61,"5,882","3,654",23.5,18.2,11.0,6.4,17.9,5.7,15.9,60.3,7.6,54.7
1,Hispanic,"2,096",7.244277e+06,4.021419e+07,8.430262e+10,18.0,0.075629,0.131855,0.57,"2,096","3,654",10.1,10.0,5.4,5.8,16.6,3.1,15.4,49.7,3.1,47.6
2,Other,"2,644",3.038650e+06,1.835358e+07,4.854390e+10,16.6,0.043549,0.060178,0.72,"2,644","3,654",9.2,7.2,3.8,4.6,8.2,1.7,6.1,13.2,2.5,10.4
3,White,"3,570",4.970512e+07,2.022911e+08,7.222635e+11,24.6,0.647950,0.663275,0.98,"3,570","3,654",16.0,8.2,5.3,3.1,7.2,2.1,5.9,112.3,20.7,87.3
0,All,"3,654",7.465616e+07,3.049883e+08,1.114690e+12,24.5,1.000000,1.000000,1.00,"3,654","3,654",15.9,9.8,6.0,4.0,10.0,2.7,8.6,235.5,33.8,200.0


## Tables

In [ ]:
all.to_csv(r'all_debt_ubi')

In [ ]:
race.to_csv(r'race_debt_ubi')

In [ ]:
education.to_csv(r'education_debt_ubi')

In [ ]:
income.to_csv(r'income_debt_ubi')

In [ ]:
age.to_csv(r'age_debt_ubi')

In [ ]:
networth.to_csv(r'networth_debt_ubi')

In [ ]:
poor.to_csv(r'poor_debt_ubi')

## Change in Gini

In [ ]:
start_gini = mdf.gini(scf, 'networth_pp', w='wgt_numper')
start_gini

0.8580552533769346

In [ ]:
ubi_gini = mdf.gini(scf, 'ubi_networth_pp', w='wgt_numper')
ubi_gini

0.8482138088314078

In [ ]:
ed_gini = mdf.gini(scf, 'no_debt_networth_pp', w='wgt_numper')
ed_gini

0.8444108755674202

In [ ]:
start_gini_inc = mdf.gini(scf, 'income_pp', w='wgt_numper')
start_gini_inc

0.5674710044436

In [ ]:
ubi_gini_inc = mdf.gini(scf, 'ubi_income_pp', w='wgt_numper')
ubi_gini_inc

0.5246819551763151

In [ ]:
no_debt_gini_inc = mdf.gini(scf, 'no_debt_income_pp', w='wgt_numper')
no_debt_gini_inc

0.5588274060412531